# 台股選股清單  
## 選股條件  
* KD值低於20，交易量大於500張，本益比小於16倍

In [ ]:
import os
from datetime import date
from datetime import datetime

import pandas as pd

In [ ]:
from finlab import data
import finlab

In [ ]:
# 引用自建公用模組
from proj_util_pkg.settings import ProjEnvSettings
from proj_util_pkg.finlab_api import finlab_manager as flm
from proj_util_pkg.google_api import gspread_manager as gsm
from proj_util_pkg.common import tw_stock_topic as tst

## 公用參數設定

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

In [ ]:
# 資訊輸出Google SpreadSheet 表單參數設定
GSPERAD_SHEET_KEY = os.environ.get('gspread_wb_key')  # Google SpreadSheet 表單ID
OUTPUT_GSHEET_NAME = '選股清單01'

In [ ]:
# 本地報表輸出路徑
REPORT_PATH = os.environ.get('report_path')

## 外部資料讀取

In [ ]:
# 讀取台股收盤價資訊
close = data.get("price:收盤價", save_to_storage=True)
vol = data.get("price:成交股數", save_to_storage=True)
stock_info = data.get('company_basic_info', save_to_storage=True)
pe_ratio = data.get('price_earning_ratio:本益比', save_to_storage=True)
pb_ratio = data.get('price_earning_ratio:股價淨值比', save_to_storage=True)
institutional_investors_foreign = data.get('institutional_investors_trading_summary:外陸資買賣超股數(不含外資自營商)', save_to_storage=True)
institutional_investors_inv_trust = data.get('institutional_investors_trading_summary:投信買賣超股數', save_to_storage=True)
institutional_investors_dealer = data.get('institutional_investors_trading_summary:自營商買賣超股數(避險)', save_to_storage=True)

## 數據分析

In [ ]:
stock_name = stock_info[['stock_id', '公司簡稱']]
stock_name = stock_name.rename(columns={'stock_id': 'symbol'})

In [ ]:
# stock_name

In [ ]:
sma18 = close.average(18)
sma50 = close.average(50)

In [ ]:
# # 篩選連三漲
# rise18_df = sma18.rise().sustain(3).tail(1)

In [ ]:
# rise18_df

In [ ]:
# 條件篩選
df2 = sma18 > sma50

today = date.today().strftime("%Y-%m-%d")
# today = "2024-05-31"
# filtered_df2 = df2[df2.index == today]
filtered_df2 = df2.tail(1)

filtered_df2

In [ ]:
filtered_symbols = filtered_df2.columns[filtered_df2.iloc[0].values].tolist()
filtered_symbols

In [ ]:
slowk, slowd = data.indicator(
    "STOCH",
    adjust_price=False,
    resample="D",
    fastk_period=9,
    slowk_period=3,
    slowk_matype=0,
    slowd_period=3,
    slowd_matype=0,
    save_to_storage=True,
)

In [ ]:
# 選股策略： 最近一個交易日，ＫＤ值低於２０且成交量大於５００張，本益比小於１６
kd_low_df = (slowd < 20) & (slowk < 20) & (vol >= 500000) & (pe_ratio <= 16)
kd_low_df = kd_low_df.tail(1)

filtered_symbols = kd_low_df.columns[kd_low_df.iloc[0]].tolist()
df_filtered_symbols = pd.DataFrame(filtered_symbols, columns=['symbol'])

df_filtered_symbols

In [ ]:
# Assuming you have a DataFrame called stock_name with columns stock_id and stock_name

# Merge df_filtered_symbols with stock_name on stock_id
merged_df = df_filtered_symbols.merge(stock_name, on='symbol')

# Print the merged DataFrame
print(merged_df)

In [ ]:
# 取得篩選股票的本益比
# Step 1: Extract symbols from merged_df
symbols = merged_df['symbol'].tolist()

# Step 2: Filter pe_ratio DataFrame using symbols
filtered_pe_ratio = pe_ratio[symbols]

# Step 3: Retrieve the latest value for each symbol
latest_pe_ratio = filtered_pe_ratio.iloc[-1]

# Print the latest pe_ratio values
print(latest_pe_ratio)


In [ ]:
df = pd.DataFrame(latest_pe_ratio).reset_index()
df.columns = ['symbol', 'pe_ratio']

# Merge df_filtered_symbols with stock_name on stock_id
merged_df = merged_df.merge(df, on='symbol')
merged_df.rename(
    columns={
        'symbol': '股票代號', 
        'pe_ratio': '本益比'
    },
    inplace=True
)
merged_df["web_link"] = merged_df["股票代號"].apply(lambda x: f"https://www.wantgoo.com/stock/{x}/technical-chart")
merged_df["題材概念股"] = merged_df["股票代號"].apply(lambda x: tst.read_topic_stocks(x))

merged_df

## 輸出結果至Google sheet

In [ ]:
# Google SpreadSheet 公用程式初始化
gspread_mgr = gsm.GspreadManager()
gspread_wb = gspread_mgr.get_spreadsheet(GSPERAD_SHEET_KEY)

print(f"更新Google 表單：{gspread_wb.title}，工作表：{OUTPUT_GSHEET_NAME}")

In [ ]:
# 刪除再重建工作表
gspread_mgr.recreate_worksheet(GSPERAD_SHEET_KEY, OUTPUT_GSHEET_NAME)

In [ ]:
# 更新工作表資料
gspread_mgr.update_worksheet_values(
    GSPERAD_SHEET_KEY, 
    OUTPUT_GSHEET_NAME, 
    [merged_df.columns.values.tolist()] + merged_df.values.tolist()
)

In [ ]:
# Update 報表更新日期資訊
now_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Update the cell with the current datetime
info_sheet = gspread_mgr.get_worksheet(GSPERAD_SHEET_KEY, "更新日誌")
info_sheet.update_cell(1, 2, now_str)